# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [6]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [7]:
w = data[data.race=='w']
print("The number of white-sounding resumes is", len(w))

# Number of callbacks for white-sounding names
print("The number of callbacks for white-sounding names is", sum(data[data.race=='w'].call))

# Rate of White Callback
w_rate = sum(data[data.race=='w'].call) / len(w)
print ("The rate of white callback is", w_rate)

The number of white-sounding resumes is 2435
The number of callbacks for white-sounding names is 235.0
The rate of white callback is 0.09650924024640657


In [8]:
b = data[data.race=='b']
print("The number of black-sounding resumes is", len(b))

# Number of callbacks for black-sounding names
print("The number of callbacks for black-sounding names is", sum(data[data.race=='b'].call))

# Rate of Black Callback
b_rate = sum(data[data.race=='b'].call) / len(b)
print ("The rate of black callback is", b_rate)

The number of black-sounding resumes is 2435
The number of callbacks for black-sounding names is 157.0
The rate of black callback is 0.06447638603696099


In [9]:
# Checking for duplicate observations:
data.duplicated().sum()

0

In [10]:
# Checking for missing values in race and call columns.
data[['race','call']].isnull().sum()

race    0
call    0
dtype: int64

<div class="span5 alert alert-success">
<p> Your answers to Q1 and Q2 here </p>
</div>

## 1. What test is appropriate for this problem? Does CLT apply?

The data is independent and identically distributed, as defined in the problem statement. We can apply central limit theorem (CLT) since there are more than 30 observations in the data. 
A two-proportion z-test is appropriate for this problem. Given that we want to compare two sample proportions with known variances the best test is the z-test.

## 2. What are the null and alternate hypotheses?


#### Null Hypothesis: 
The proportion of callbacks for white-sounding names and the proportion of callbacks for black-sounding names are equal.

#### Alternative Hypothesis: 
The proportion of callbacks for white-sounding names and the proportion of callbacks for black-sounding names are not equal.

In [11]:
# Your solution to Q3 here

## 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

#### a. Bootstrapping Approach

In [12]:
# Calculating margin of error, confidence interval and p-value using bootstrapping approach
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data1, data2, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(shape=size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data1, func) - bootstrap_replicate_1d(data2, func)

    return bs_replicates

In [13]:
# Draw bootstrap replicates of the mean difference between the frequencies of calls for resumes 
# for black-sounding and white-sounding names: bs_replicates

bs_replicates = draw_bs_reps(w.call, b.call, np.mean, 10000)

# Compute the 95% confidence interval: conf_int
conf_int = np.percentile(bs_replicates, [2.5, 97.5])


In [14]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(shape=size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

In [15]:
# Compute difference of mean impact force from experiment: empirical_diff_means
empirical_diff_means = diff_of_means(w.call, b.call)

# Under the null hypothesis there is no difference between the frequency of the calls between white-sounding 
# and black-sounding names.
# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(w.call, b.call,
                                 diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

print('Statistical inference using the bootstrap approach')
# Print the result
print('The p-value:', p)

# Also print the confidance interval value
print('The 95% confidence interval:', conf_int)

Statistical inference using the bootstrap approach
The p-value: 0.0
The 95% confidence interval: [0.01683778 0.0476386 ]


Conclusion: p_value is less than 0.05. Null Hypothesis will be rejected. We can observe that race has a significant impact on the rate of callbacks for resumes.

#### b. Frequentist Statical Approach

#### z_score

In [16]:
w_call = np.sum(data[data.race=='w'].call)
b_call = np.sum(data[data.race=='b'].call)          
phat = (w_call + b_call) / (len(w) + len(b))
z = (empirical_diff_means) / np.sqrt(phat * (1 - phat) * (( 1./ len(w)) + (1./len(b))))
print ('z score:', z)

z score: 4.108412235238472


#### p_value

In [17]:
p = stats.norm.cdf(-z)*2
print ('p-value:',p)

p-value: 3.983885409504418e-05


#### Margin of Error

In [18]:
moe = 1.96 * np.sqrt(phat * (1 - phat) * (( 1./len(w)) + (1./len(b))))
print('Margin of Error:', moe)

Margin of Error: 0.015281912310894095


#### Confidence Interval

In [19]:
ci = empirical_diff_means + np.array([-1, 1]) * moe
print('Confidence interval:', ci)

Confidence interval: [0.01675094 0.04731477]


#### Conclusion: 
The p-value is very low than 5%, so the null hypothesis is rejected. There is evidence that, there is difference in the proportion of callbacks for white-sounding names and the proportion of callbacks for black-sounding names.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## 4. Write a story describing the statistical significance in the context or the original problem.

We performed a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes. That analysis tested the effect of race on callback success, and the statistics test indicates that racial discrimination in the United States labor market seems significant. Given our sample data and our findings in this project, we can observe that a black- or white-sounding name on a resume has a significant effect on callbacks.


Based on the result of p-value which is too close to 0. This means that the effect that we see (a difference in the proportion of callbacks between white and black sounding names) is significant. There is a 95% chance that the true difference of white-sounding call back rates and black-sounding call back rates is between 0.016 and 0.04. This means we are 95% confident that there exists a difference between the two races in which individuals with white-sounding names are favored and called back more often.

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

According to the results above, we can observe that a black- or white-sounding name on a resume has a significant effect on callbacks. This analysis does not mean the race/name is the most important factor in callback success because there are more other factors that may affect employers' decision, like 'year of experience', 'education', 'resume quality', 'background' , 'different types of jobs', etc.